# HW2. Kaggle Competition

In [8]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split

In [9]:
class TitanicDataset(Dataset):
  def __init__(self, X, y):
    self.X = torch.FloatTensor(X)
    self.y = torch.LongTensor(y)

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    feature = self.X[idx]
    target = self.y[idx]
    return {'input': feature, 'target': target}

  def __str__(self):
    str = "Data Size: {0}, Input Shape: {1}, Target Shape: {2}".format(
      len(self.X), self.X.shape, self.y.shape
    )
    return str


사용자 정의 데이터셋
- TitanicDataset 클래스
    - PyTorch의 'Dataset' 클래스를 상속하며 훈련 데이터를 로드하고 전처리하는데 사용
 
    - __init__ 메서드 :
        - 데이터셋을 초기화하는 역할
        - X(입력 데이터)와 y(대상 레이블)로 전달
    - __len__ 메서드 :
        - 데이터셋의 길이를 반환하는 메서드
        - 데이터 포인터의 수를 반환
    - __getitem__ 메서드 :
        - 데이터셋에서 주어진 인덱스 'idx'에 해당하는 데이터 포인트 반환
        - idx에 해당하는 X의 feature와 y의 target 값을 딕셔너리 형태로 반환
    - __str__ 메서드:
        - 데이터셋을 문자열로 표현할 때 사용
        - 데이터셋 요약 정보 표시에 유용

In [10]:
class TitanicTestDataset(Dataset):
  def __init__(self, X):
    self.X = torch.FloatTensor(X)

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    feature = self.X[idx]
    return {'input': feature}

  def __str__(self):
    str = "Data Size: {0}, Input Shape: {1}".format(
      len(self.X), self.X.shape
    )
    return str

- TitanicTestDataset 클래스
    - PyTorch의 'Dataset' 클래스를 상속하며 훈련 테스트 데이터를 로드하고 전처리하는데 사용
 
    - __init__ 메서드 :
        - 테스트 데이터셋을 초기화하는 역할
        - X(입력 데이터)를 초기화
    - __len__ 메서드 :
        - 데이터셋 길이 반환
    - __getitem__ 메서드 :
        - 데이터셋에서 주어진 인덱스 'idx'에 해당하는 데이터 포인트 반환
        - 'self.X[idx]'의 입력데이터 중 idx에 해당하는 데이터 포인트 반환
    - __str__ 메서드 :
        - 데이터셋의 크기와 입력데이터의 형태를 포함하는 문자열 반환
<br>

- TitanicDataset 과 TitanicTestDataset 클래스의 차이
    - TitanicDataset : 훈련 데이터에 사용되며 입력데이터, target을 모두 처리함
    - TitanicTestDataset : 테스트 데이터에 사용, 입력데이터만 처리

In [11]:
def get_preprocessed_dataset():

    train_data_path = os.path.join(".", "data", "train.csv")
    test_data_path = os.path.join(".", "data", "test.csv")

    train_df = pd.read_csv(train_data_path)
    test_df = pd.read_csv(test_data_path)

    all_df = pd.concat([train_df, test_df], sort=False)

    all_df = get_preprocessed_dataset_1(all_df)

    all_df = get_preprocessed_dataset_2(all_df)

    all_df = get_preprocessed_dataset_3(all_df)

    all_df = get_preprocessed_dataset_4(all_df)

    all_df = get_preprocessed_dataset_5(all_df)

    all_df = get_preprocessed_dataset_6(all_df)

    train_X = all_df[~all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)
    train_y = train_df["Survived"]

    test_X = all_df[all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)

    dataset = TitanicDataset(train_X.values, train_y.values)
    #print(dataset)
    train_dataset, validation_dataset = random_split(dataset, [0.8, 0.2])
    test_dataset = TitanicTestDataset(test_X.values)
    #print(test_dataset)

    return train_dataset, validation_dataset, test_dataset

- get_preprocessed_dataset 함수
    - 데이터 전처리 및 데이터셋 구성 함수
    - 데이터 파일 가져오기
        - train_data_path : 학습 데이터 train.csv 파일 경로
        - test_data_path : 테스트 데이터 test.csv 파일 경로
    - 훈련, 테스트 데이터 로드
        - pd.read_csv를 사용하여 train.csv, test.csv 데이터를 train_df, test_df 변수에 로드
    - 전체 데이터프레임 생성
        - pd.concat을 사용하여 훈련데이터, 테스트데이터를 합침
        - all_df 변수에 저장
    - 데이터 전처리 함수 적용
        -  데이터 전처리 함수 (get_preprocessed_dataset_1 ~ get_preprocessed_dataset_6)
        -  차례로 호출하여 all_df에 전처리 적용
    - 훈련, 대상 데이터 분리
        - train_df에서 target인 Survived를 분리하여 train_y 변수에 저장
        - all_df에서 Survived 열을 제거한 다른 훈련데이터를 train_X에 저장
    - 데이터셋 구성
        - TitanicDataset 클래스를 사용하여 train_X와 train_y를 기반으로 훈련 데이터셋 dataset을 생성
        - random_split 함수를 사용하여 dataset을 훈련 및 검증 데이터셋으로 분할. 여기서 80%는 훈련 데이터로, 나머지 20%는 검증 데이터로 구성됨
        - TitanicTestDataset 클래스를 사용하여 test_X를 기반으로 테스트 데이터셋 test_dataset을 생성
    - 데이터셋 반환
        - train_dataset, validation_dataset, 및 test_dataset 반환

### 데이터 가공

In [12]:
def get_preprocessed_dataset_1(all_df):
    # Pclass별 Fare 평균값을 사용하여 Fare 결측치 메우기
    Fare_mean = all_df[["Pclass", "Fare"]].groupby("Pclass").mean().reset_index()
    Fare_mean.columns = ["Pclass", "Fare_mean"]
    all_df = pd.merge(all_df, Fare_mean, on="Pclass", how="left")
    all_df.loc[(all_df["Fare"].isnull()), "Fare"] = all_df["Fare_mean"]

    return all_df

- 승객 등급별로 운임료의 평균값을 계산 ->이 값을 사용하여 Fare의 결측치를 메우는 역할

In [13]:
def get_preprocessed_dataset_2(all_df):
    # name을 세 개의 컬럼으로 분리하여 다시 all_df에 합침
    name_df = all_df["Name"].str.split("[,.]", n=2, expand=True)
    name_df.columns = ["family_name", "honorific", "name"]
    name_df["family_name"] = name_df["family_name"].str.strip()
    name_df["honorific"] = name_df["honorific"].str.strip()
    name_df["name"] = name_df["name"].str.strip()
    all_df = pd.concat([all_df, name_df], axis=1)

    return all_df

- 승객의 이름 열을 분석하여 세 가지 컬럼으로 분리
    - family_name, honorific, name 컬럼

In [14]:
def get_preprocessed_dataset_3(all_df):
    # honorific별 Age 평균값을 사용하여 Age 결측치 메우기
    honorific_age_mean = all_df[["honorific", "Age"]].groupby("honorific").median().round().reset_index()
    honorific_age_mean.columns = ["honorific", "honorific_age_mean", ]
    all_df = pd.merge(all_df, honorific_age_mean, on="honorific", how="left")
    all_df.loc[(all_df["Age"].isnull()), "Age"] = all_df["honorific_age_mean"]
    all_df = all_df.drop(["honorific_age_mean"], axis=1)

    return all_df

- 승객의 honorific에 따라 Age의 결측치를 메우는 역할
    - 호칭별로 나이의 중앙값 계산하여 데이터프레임 생성 -> honorific_age_mean
    - 결측치가 있는 행의 나이를 해당 호칭의 중앙값으로 채워서 결측치 처리

In [15]:
def get_preprocessed_dataset_4(all_df):
    # 가족수(family_num) 컬럼 새롭게 추가
    all_df["family_num"] = all_df["Parch"] + all_df["SibSp"]

    # 혼자탑승(alone) 컬럼 새롭게 추가
    all_df.loc[all_df["family_num"] == 0, "alone"] = 1
    all_df["alone"].fillna(0, inplace=True)

    # 학습에 불필요한 컬럼 제거
    all_df = all_df.drop(["PassengerId", "Name", "family_name", "name", "Ticket", "Cabin"], axis=1)

    return all_df

- 가족 수와 혼자 탑승 여부의 특성 생성
- 가족 수는 Parch, SibSp를 합한 값
    - 혼자 탑승 여부는 가족 수가 0인 경우 1로 표시
    - 그렇지 않은 경우는 0으로 표시
- 학습에 불필요한 컬럼은 데이터프레임에서 제거

In [16]:
def get_preprocessed_dataset_5(all_df):
    # honorific 값 개수 줄이기
    all_df.loc[
    ~(
            (all_df["honorific"] == "Mr") |
            (all_df["honorific"] == "Miss") |
            (all_df["honorific"] == "Mrs") |
            (all_df["honorific"] == "Master")
    ),
    "honorific"
    ] = "other"
    all_df["Embarked"].fillna("missing", inplace=True)

    return all_df

- honorific의 값을 정리하고 Mr, Miss, Mrs, Master 이외의 값은 other로 변경
- Embarked 열의 결측치를 missing으로 채워서 카테고리 변수 처리

In [17]:
def get_preprocessed_dataset_6(all_df):
    # 카테고리 변수를 LabelEncoder를 사용하여 수치값으로 변경하기
    category_features = all_df.columns[all_df.dtypes == "object"]
    from sklearn.preprocessing import LabelEncoder
    for category_feature in category_features:
        le = LabelEncoder()
        if all_df[category_feature].dtypes == "object":
          le = le.fit(all_df[category_feature])
          all_df[category_feature] = le.transform(all_df[category_feature])

    return all_df

- LableEncoder를 사용하여 수치값으로 변환
    - 데이터프레임의 문자열 형식의 카테고리 변수를 수치형으로 변환
    - 머신러닝 모델에 입력할 수 있는 데이터로 변환

In [25]:
import wandb
from torch import nn
from datetime import datetime

In [26]:
wandb.login()

True

In [27]:
class MyModel(nn.Module):
  def __init__(self, n_input, n_output):
    super().__init__()

    self.model = nn.Sequential(
            nn.Linear(n_input, 30),
            nn.RReLU(),
            nn.Linear(30, 50),
            nn.RReLU(),
            nn.Linear(50, 100),
            nn.RReLU(),
            nn.Linear(100, 50),
            nn.RReLU(),
            nn.Linear(50, 30),
            nn.RReLU(),
            nn.Linear(30, n_output),
        )

  def forward(self, x):
    x = self.model(x)
    return x

In [28]:
def test(test_data_loader):
  print("[TEST]")
  batch = next(iter(test_data_loader))
  print("{0}".format(batch['input'].shape))
  my_model = MyModel(n_input=11, n_output=2)
  output_batch = my_model(batch['input'])
  prediction_batch = torch.argmax(output_batch, dim=1)
  for idx, prediction in enumerate(prediction_batch, start=892):
      print(idx, prediction.item())

In [29]:
def get_model_criterion_optimizer():
    my_model = MyModel(n_input=11, n_output=2)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(my_model.parameters(), lr=config["learning_rate"])

    return my_model, criterion, optimizer

In [30]:
def training_loop(my_model, criterion, optimizer, use_wandb, train_data_loader, validation_data_loader):
  
    for epoch in range(config["max_num_epoch"]):
        my_model.train()
        total_loss = 0.0
        correct = 0
        total = 0

        for idx, batch in enumerate(train_data_loader):
            optimizer.zero_grad()
            input_data = batch['input']
            target = batch['target']

            output = my_model(input_data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

        accuracy = correct / total
        average_loss = total_loss / len(train_data_loader)
        
        if use_wandb:
            wandb.log({"train_loss": average_loss, "train_accuracy": accuracy})

        print(f"Epoch [{epoch + 1}/{config['max_num_epoch']}], "
            f"Loss: {average_loss:.4f}, "
            f"Accuracy: {accuracy:.4f}")
        
        my_model.eval()
        total_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for idx, batch in enumerate(validation_data_loader):
                input_data = batch['input']
                target = batch['target']

                output = my_model(input_data)
                loss = criterion(output, target)

                total_loss += loss.item()

                _, predicted = torch.max(output.data, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()

        accuracy = correct / total
        average_loss = total_loss / len(validation_data_loader)

        if use_wandb:
            wandb.log({"val_loss": average_loss, "val_accuracy": accuracy})

        print(f"Validation Loss: {average_loss:.4f}, "
            f"Validation Accuracy: {accuracy:.4f}")

In [31]:
if __name__ == "__main__":

    ENV_NAME = "Kaggle_Titanic"
    use_wandb = True
    current_time = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')
    config = {
        "env_name": ENV_NAME,
        "max_num_epoch": 100,
        "batch_size": 16,
        "learning_rate": 0.001,
    }
    if use_wandb:
            wandb = wandb.init(
                project=config["env_name"],
                name=current_time,
                config=config
            )

    train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()

    print("train_dataset: {0}, validation_dataset.shape: {1}, test_dataset: {2}".format(
        len(train_dataset), len(validation_dataset), len(test_dataset)
    ))
    print("#" * 50, 1)

    for idx, sample in enumerate(train_dataset):
        print("{0} - {1}: {2}".format(idx, sample['input'], sample['target']))

    print("#" * 50, 2)

    train_data_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
    validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=16, shuffle=True)
    test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

    print("[TRAIN]")
    for idx, batch in enumerate(train_data_loader):
        print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

    print("[VALIDATION]")
    for idx, batch in enumerate(validation_data_loader):
        print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

    print("#" * 50, 3)

    test(test_data_loader)

    print("#" * 50, 4)

    my_model, criterion, optimizer = get_model_criterion_optimizer()

    training_loop(
        my_model = my_model,
        criterion = criterion,
        optimizer = optimizer,
        use_wandb = use_wandb,
        train_data_loader = train_data_loader,
        validation_data_loader = validation_data_loader
    )
    wandb.finish()

    final_model_path = './Model/my_model_final.pth'
    torch.save(my_model.state_dict(), final_model_path)

    my_model = MyModel(n_input=11, n_output=2)
    my_model.load_state_dict(torch.load('./Model/my_model_final.pth'))
    my_model.eval()

    # 테스트 데이터를 DataLoader에 넣습니다.
    test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

    # 모델을 사용하여 테스트 데이터에 대한 예측을 수행합니다.
    predictions = []
    for batch in test_data_loader:
        inputs = batch['input']
        outputs = my_model(inputs)
        predictions_batch = torch.argmax(outputs, dim=1)
        predictions.extend(predictions_batch.tolist())

    # submission.csv 파일을 생성합니다.
    submission_df = pd.DataFrame({
        'PassengerId': range(892, 892 + len(predictions)),
        'Survived': predictions
    })

    # CSV 파일로 저장합니다.
    submission_df.to_csv('./Output/submission.csv', index=False)

    print("Submission 파일 'submission.csv'생성")
    

train_dataset: 713, validation_dataset.shape: 178, test_dataset: 418
################################################## 1
0 - tensor([ 2.0000,  0.0000, 34.0000,  0.0000,  0.0000, 10.5000,  2.0000, 21.1792,
         3.0000,  0.0000,  1.0000]): 1
1 - tensor([ 2.0000,  1.0000, 66.0000,  0.0000,  0.0000, 10.5000,  2.0000, 21.1792,
         2.0000,  0.0000,  1.0000]): 0
2 - tensor([ 3.0000,  1.0000, 29.0000,  0.0000,  0.0000,  7.8958,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
3 - tensor([ 1.0000,  1.0000, 42.0000,  0.0000,  0.0000, 26.2875,  2.0000, 87.5090,
         2.0000,  0.0000,  1.0000]): 1
4 - tensor([ 3.0000,  0.0000, 15.0000,  0.0000,  0.0000,  7.2250,  0.0000, 13.3029,
         1.0000,  0.0000,  1.0000]): 1
5 - tensor([ 2.0000,  1.0000,  3.0000,  1.0000,  1.0000, 26.0000,  2.0000, 21.1792,
         0.0000,  2.0000,  0.0000]): 1
6 - tensor([ 3.0000,  0.0000, 22.0000,  0.0000,  0.0000,  7.7875,  1.0000, 13.3029,
         1.0000,  0.0000,  1.0000]): 1
7 - tensor([ 3.00

train_accuracy,▁▃▃▂▄▄▅▆▆▅▆▆▆▆▆▇▇▆▇▇▆▇▇▇▆▆▇▇▇█▇▇▇▇█▇▇█▇█
train_loss,█▇▆▇▅▄▄▄▃▄▃▃▃▃▃▃▃▃▂▂▃▂▂▂▃▂▂▂▂▂▂▂▂▂▁▁▁▁▂▁
val_accuracy,▁▁▁▁▁▅▅▄▄▅▅▇█▅▆▆▆▆▇▇▇▆██▇▇▆▅▂█▇▇█▃▇▆▆▆▇▆
val_loss,█▆▅▆▄▃▄▂▂▂▃▂▂▂▂▂▂▂▁▁▂▂▁▄▂▁▃▃▃▂▁▂▁▄▂▃▂▂▃▂
train_accuracy,0.84292
train_loss,0.36808
val_accuracy,0.82584
val_loss,0.37943


Submission 파일 'submission.csv'생성


"![Titanic_kaggle_result](./Titanic_kaggle_result.PNG)"

# 학습후기

- PyTorch를 사용하여 기계 학습 모델을 구축하고 학습시키는 작업을 하였습니다.
- 이를 통해 학습한 내용
    - 데이터 전처리와 준비 : 데이터 전처리를 수행하는 함수와 클래스를 활용해보았습니다.
        - get_preprocessed_dataset을 활용하여 6번의 가공을 거쳐 데이터 처리
    - 학습 및 검증루프 :
        - traing_loop함수를 통해 모델을 학습하고 검증하는 과정을 구현함.
        - 손실함수, optimizer 및 epoch 수를 설정하여 학습 및 검증 데이터에 대한 손실과 정확도를 기록함.
    - 결과 저장 및 예측 :
        - 학습 완료 후 최종 모델의 상태를 저장하고, 저장된 모델을 로드하여 CSV 파일로 제출 파일을 생성함.
    - wandb 사용 :
        - 학습 과정과 결과를 기록하고 시각화함.

- 이번 과제를 통해 머신러닝을 이해하고 PyTorch를 사용한 신경망 모델 구축 경험을 할 수 있었습니다.
- 데이터 처리, 모델 조정, 학습 및 평가 과정을 이해하고 사용할 수 있었습니다.